In [25]:
import min_heap
import random
import matplotlib.pyplot as plt

class DirectedWeightedGraph:

    def __init__(self):
        self.adj = {}
        self.weights = {}

    def are_connected(self, node1, node2):
        for neighbour in self.adj[node1]:
            if neighbour == node2:
                return True
        return False

    def adjacent_nodes(self, node):
        return self.adj[node]

    def add_node(self, node):
        self.adj[node] = []

    def add_edge(self, node1, node2, weight):
        if node2 not in self.adj[node1]:
            self.adj[node1].append(node2)
        self.weights[(node1, node2)] = weight

    def w(self, node1, node2):
        if self.are_connected(node1, node2):
            return self.weights[(node1, node2)]

    def number_of_nodes(self):
        return len(self.adj)


def dijkstra(G, source):
    pred = {} #Predecessor dictionary. Isn't returned, but here for your understanding
    dist = {} #Distance dictionary
    Q = min_heap.MinHeap([])
    nodes = list(G.adj.keys())

    #Initialize priority queue/heap and distances
    for node in nodes:
        Q.insert(min_heap.Element(node, float("inf")))
        dist[node] = float("inf")
    Q.decrease_key(source, 0)

    #Meat of the algorithm
    while not Q.is_empty():
        current_element = Q.extract_min()
        current_node = current_element.value
        dist[current_node] = current_element.key
        for neighbour in G.adj[current_node]:
            if dist[current_node] + G.w(current_node, neighbour) < dist[neighbour]:
                Q.decrease_key(neighbour, dist[current_node] + G.w(current_node, neighbour))
                dist[neighbour] = dist[current_node] + G.w(current_node, neighbour)
                pred[neighbour] = current_node
    return dist


def bellman_ford(G, source):
    pred = {} #Predecessor dictionary. Isn't returned, but here for your understanding
    dist = {} #Distance dictionary
    nodes = list(G.adj.keys())

    #Initialize distances
    for node in nodes:
        dist[node] = float("inf")
    dist[source] = 0

    #Meat of the algorithm
    for _ in range(G.number_of_nodes()):
        for node in nodes:
            for neighbour in G.adj[node]:
                if dist[neighbour] > dist[node] + G.w(node, neighbour):
                    dist[neighbour] = dist[node] + G.w(node, neighbour)
                    pred[neighbour] = node
    return dist


def total_dist(dist):
    total = 0
    for key in dist.keys():
        total += dist[key]
    return total

def create_random_complete_graph(x,y):
    G = DirectedWeightedGraph()
    for i in range(x):
        G.add_node(i)
    for i in range(x):
        for j in range(x):
            if i != j:
                G.add_edge(i, j, random.randint(1, y))
    return G


#Assumes G represents its nodes as integers 0,1,...,(n-1)
def mystery(G):
    n = G.number_of_nodes()
    d = init_d(G)
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if d[i][j] > d[i][k] + d[k][j]: 
                    d[i][j] = d[i][k] + d[k][j]
    return d

def init_d(G):
    n = G.number_of_nodes()
    d = [[float("inf") for j in range(n)] for i in range(n)]
    for i in range(n):
        for j in range(n):
            if G.are_connected(i, j):
                d[i][j] = G.w(i, j)
        d[i][i] = 0
    return d
def path_from_pred(pred,start_node,end_node):
    path = []
    current_node = end_node
    while current_node != start_node:
        path.append(current_node)
        current_node = pred[current_node]
    path.append(current_node)
    path.reverse()
    return path



In [31]:
import csv
station_line = {}
direct_dis = {}
path_length = {}


def add_different_value(list,v1,v2):
    # This function check if v1 and v2 are in the list, if not then add them to list.
    if not v1 in list:
        list.append(v1)
    if not v2 in list:
        list.append(v2)

with open('london_stations.csv', mode ='r')as file:
    ls_csv = csv.reader(file)
    for line in ls_csv:
        if line[0] == "id":
            continue
        direct_dis[int(line[0])] = (float(line[1]),float(line[2]),int(line[6]))

with open("london_connections.csv", mode = 'r') as file:
    lc_csv = csv.reader(file)
    
    for line in lc_csv:
        if line[0] == "station1":
            continue
        if int(line[2]) not in station_line.keys():
            station_line[int(line[2])] = []
        add_different_value(station_line[int(line[2])],int(line[0]),int(line[1]))
        path_length[(int(line[0]),int(line[1]))] = int(line[3])
        path_length[(int(line[1]),int(line[0]))] = int(line[3])


def h(s,d):
    return ((direct_dis[s][0] - direct_dis[d][0])**2 + (direct_dis[s][1] - direct_dis[d][1])**2)**(1/2)

def a_star(G, s, d, h):
    pred = {} #Predecessor dictionary. Isn't returned, but here for your understanding
    dist = {} #Distance dictionary
    Q = min_heap.MinHeap([])
    nodes = list(G.adj.keys())
    for node in nodes:
        Q.insert(min_heap.Element(node, float("inf")))
        dist[node] = float("inf")
    Q.decrease_key(s, h(s,d))   
    while not Q.is_empty():
        current_element = Q.extract_min()
        if current_element == d:
            break
        current_node = current_element.value
        dist[current_node] = current_element.key - h(current_node,d)
        for neighbour in G.adj[current_node]:
            if dist[current_node] + G.w(current_node, neighbour) < dist[neighbour]:
                Q.decrease_key(neighbour, dist[current_node] + G.w(current_node, neighbour) + h(neighbour,d) )
                dist[neighbour] = dist[current_node] + G.w(current_node, neighbour)
                pred[neighbour] = current_node
    return (pred, path_from_pred(pred,s,d))

#depand on the 2 csv file create a directed weighted graph
def create_london_graph():
    london_graph = DirectedWeightedGraph()
    for key in direct_dis.keys():
        london_graph.add_node(key)
    for key in path_length.keys():
        london_graph.add_edge(key[0],key[1],path_length[key])
    #station_line is a dictionary, key is the line number, value is a list of station id
    return london_graph



G = create_london_graph()





In [52]:
# find all one transfer station_line from s 
def find_one_transfer_station(s):
    one_transfer_station = []
    for line in station_line.keys():
        if s in station_line[line]:
            for station in station_line[line]:
                if station != s:
                    one_transfer_station.append(station)
    return one_transfer_station
print(find_one_transfer_station(1))



[3, 263, 295, 15, 78, 269, 17, 110, 293, 18, 186, 193, 21, 67, 25, 161, 255, 33, 36, 164, 289, 44, 166, 52, 265, 66, 85, 72, 73, 74, 99, 122, 138, 287, 270, 80, 205, 231, 83, 129, 87, 285, 96, 195, 236, 108, 141, 209, 268, 213, 244, 200, 242, 229, 273, 300, 248, 267, 299, 73, 234, 265, 5, 194, 252, 9, 31, 232, 10, 95, 128, 17, 74, 110, 30, 176, 190, 303, 39, 145, 57, 187, 60, 126, 151, 182, 99, 75, 210, 222, 160, 236, 107, 133, 197, 116, 117, 118, 132, 125, 134, 271, 223, 130, 131, 146, 220, 266, 235, 251]


In [57]:
def one_transfer_routes(G, source, station_line):
    neighbors = G.adjacent_nodes(source)
    routes = {}
    for n in neighbors:
        # find lines that connect source to n
        lines = []
        for line, stations in station_line.items():
            if source in stations and n in stations and source != n: # exclude source station's own line
                lines.append(line)
        routes[n] = lines
    
    transfer_stations = []
    for n in neighbors:
        for line in station_line.keys():
            if n in station_line[line] and line not in routes.values():
                for station in station_line[line]:
                    if station != n and station not in neighbors:
                        transfer_stations.append(station)
    
    for t in transfer_stations:
        for n in G.adjacent_nodes(t):
            if n != source and n not in neighbors:
                # find lines that connect source to n
                lines = []
                for line, stations in station_line.items():
                    if source in stations and n in stations:
                        lines.append(line)
                if n in routes:
                    routes[n].extend(lines)
                else:
                    routes[n] = lines
    
    return routes
print(one_transfer_routes(G, 1, station_line))

{52: [4], 73: [4, 10], 234: [10], 265: [4, 10], 263: [4, 4, 4, 4, 4, 4], 295: [4, 4, 4, 4, 4, 4], 156: [], 2: [], 166: [4, 4, 4, 4, 4, 4], 3: [4, 4, 4, 4, 4, 4], 244: [4, 4, 4, 4, 4, 4], 225: [], 228: [], 78: [4, 4, 4, 4, 4, 4], 269: [4, 4, 4, 4, 4, 4], 15: [4, 4, 4, 4, 4, 4], 270: [4, 4, 4, 4, 4, 4], 21: [4, 4, 4, 4, 4, 4], 110: [4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10], 293: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], 74: [4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10], 17: [4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10, 4, 10], 209: [4, 4, 4, 4, 4, 4, 4, 4, 4], 101: [], 186: [4, 4, 4, 4, 4, 4], 193: [4, 4, 4, 4, 4, 4], 127: [], 208: [], 18: [4, 4, 4, 4, 4, 4], 122: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], 82: [], 278: [], 83: [4, 4, 4], 218: [], 67: [4, 4, 4, 4, 4, 4], 66: [4, 4, 4, 4, 4, 4],

可以通过一次换乘到达的站点及线路组合:
1(Line 4) -> 78(Line 4) -> 3
1(Line 4) -> 78(Line 4) -> 263
1(Line 4) -> 78(Line 4) -> 295
1(Line 4) -> 78(Line 4) -> 15
1(Line 4) -> 78(Line 4) -> 269
1(Line 4) -> 78(Line 4) -> 17
1(Line 4) -> 78(Line 4) -> 110
1(Line 4) -> 78(Line 4) -> 293
1(Line 4) -> 78(Line 4) -> 18
1(Line 4) -> 78(Line 4) -> 186
1(Line 4) -> 78(Line 4) -> 193
1(Line 4) -> 78(Line 4) -> 21
1(Line 4) -> 78(Line 4) -> 67
1(Line 4) -> 78(Line 4) -> 25
1(Line 4) -> 78(Line 4) -> 161
1(Line 4) -> 78(Line 4) -> 255
1(Line 4) -> 78(Line 4) -> 33
1(Line 4) -> 78(Line 4) -> 36
1(Line 4) -> 78(Line 4) -> 164
1(Line 4) -> 78(Line 4) -> 289
1(Line 4) -> 78(Line 4) -> 44
1(Line 4) -> 78(Line 4) -> 166
1(Line 4) -> 78(Line 4) -> 52
1(Line 4) -> 78(Line 4) -> 265
1(Line 4) -> 78(Line 4) -> 66
1(Line 4) -> 78(Line 4) -> 85
1(Line 4) -> 78(Line 4) -> 72
1(Line 4) -> 78(Line 4) -> 73
1(Line 4) -> 78(Line 4) -> 74
1(Line 4) -> 78(Line 4) -> 99
1(Line 4) -> 78(Line 4) -> 122
1(Line 4) -> 78(Line 4) -> 138
1(Line